# Unit simplification
Problems in unit simplification

In [61]:
from __future__ import print_function, division
import tellurium as te
r = te.loada("""
// Units
unit mmol = 1e-3 mole;
unit mM = mmol / litre;
unit mmol_per_s = mmol / second;
unit test = 1000 litre / metre^3

substance = mmol;
volume = litre;
area = metre^2; 
length = metre;
extent = mmol;
time_unit = second;

// Species
species S1 = 0.0

// Reaction
J0: S1 ->; v0 * k1;
v0 = 1.0 mmol_per_s;
k1 = 1.0 test;

""")
sbml_str = r.getSBML()
print(sbml_str)

<?xml version="1.0" encoding="UTF-8"?>
<!-- Created by libAntimony version v2.8.1 on 2016-02-16 12:04 with libSBML version 5.12.1. -->
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model id="__main" name="__main" substanceUnits="substance" timeUnits="time_unit" volumeUnits="volume" areaUnits="area" lengthUnits="length" extentUnits="extent">
    <listOfUnitDefinitions>
      <unitDefinition id="mmol">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="mM">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
          <unit kind="litre" exponent="-1" scale="0" multiplier="1"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition id="mmol_per_s">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
          <unit kind="second" exponent="-1" scale

In [59]:
# validate
from multiscale.sbmlutils import validation
import tempfile
import libsbml
doc = libsbml.readSBML(sbml_str)
f = tempfile.NamedTemporaryFile(suffix='.xml')
f.write(sbml_str)
f.flush()
validation.check_sbml(f.name)


 read time (ms): 0.002287
 validation error(s): 1

line 75: (10541 [Warning]) The units of the 'math' formula in a <kineticLaw> definition are expected to be the equivalent of _substance per time_.
Reference: L3V1 Sections 4.11.7, 4.2.4 AND 4.9.4
 In level 3 the expected units are extent_per_time. Expected units are mole (exponent = 1, multiplier = 1, scale = -3), second (exponent = -1, multiplier = 1, scale = 0) but the units returned by the <math> expression in the <kineticLaw> (from the <reaction> with id 'J0') are mole (exponent = 1, multiplier = 1, scale = -3), second (exponent = -1, multiplier = 1, scale = 0), litre (exponent = 1, multiplier = 1, scale = 3), metre (exponent = -3, multiplier = 1, scale = 0).



1